In [1]:
# repo url(https://github.com/xwang71785/buffett.git)
# 只针对中国大陆股票市场
# 仅限于概念验证，勿用于实际投资中
# QQ：2908683294
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
import pandas_datareader.data as web
import datetime as dt
import matplotlib.dates as mdates

#### 1.估值

In [3]:
# 基于网易构造三张财务报表的数据源URL
server = "http://quotes.money.163.com/"
path = "service/"
table1 = "zcfzb_"
table2 = "lrb_"
table3 = "xjllb_"
stock = "600585"    # 根据需求变更stock中的股票代码
type = ".html?type=year"

target1 = server + path + table1 + stock + type
target2 = server + path + table2 + stock + type
target3 = server + path + table3 + stock + type

print(target1, '\n', target2, '\n', target3)

http://quotes.money.163.com/service/zcfzb_600585.html?type=year 
 http://quotes.money.163.com/service/lrb_600585.html?type=year 
 http://quotes.money.163.com/service/xjllb_600585.html?type=year


In [4]:
# 获取三张财务报表CSV格式原始数据
df1 = pd.read_csv(target1, header=0, prefix="V", encoding="gb18030")
df2 = pd.read_csv(target2, header=0, prefix="V", encoding="gb18030")
df3 = pd.read_csv(target3, header=0, prefix="V", encoding="gb18030")

In [5]:
# 报表行列转置，方便进行矢量运算
bs = df1.transpose()
pl = pd.DataFrame(df2.values.T, columns=df2['报告日期'], index=df2.columns)
cf = pd.DataFrame(df3.values.T, columns=df3[' 报告日期'], index=df2.columns)

In [6]:
bs.drop('报告日期', inplace=True)    # 删除第一行
bs.drop(bs.tail(1).index, inplace=True)    # 删除最后一行

In [7]:
# 网易提供的CSV文件是字符型数据，无法直接进行算术元算。将字符型数据转换成数值型
headers_bs = bs.columns
headers_pl = pl.columns
headers_cf = cf.columns

for head in headers_bs:
    bs[head] = pd.to_numeric(bs[head], errors='coerce').fillna(0) # 不符合转换条件的设为‘Nan’，再用0替换

for head in headers_pl:
    pl[head] = pd.to_numeric(pl[head], errors='coerce').fillna(0)

for head in headers_cf:
    cf[head] = pd.to_numeric(cf[head], errors='coerce').fillna(0)

In [8]:
# 整理资产负债表
bs_sum = pd.DataFrame()    # 利用column的数字标识进行分类汇总

bs_sum['operat_current'] = bs[0]+bs[6]+bs[7]+bs[8]+bs[12]+bs[13]+bs[14]+bs[15]+bs[17]+bs[20]+bs[21]+bs[22]+bs[23]+bs[19]
bs_sum['operat_non_curr'] = bs[29]+bs[28]+bs[36]+bs[37]+bs[38]+bs[39]+bs[40]+bs[43]+bs[44]+bs[45]+bs[46]+bs[42]+bs[49]+bs[48]
bs_sum['operat_liability'] = bs[57]+bs[60]+bs[59]+bs[64]+bs[63]+bs[67]+bs[66]+bs[70]+bs[71]+bs[72]+bs[69]+bs[78]+bs[79]+bs[80]+bs[77]+bs[82]+bs[87]+bs[91]+bs[90]
bs_sum['financial_assets'] = bs[2]+bs[3]+bs[4]+bs[5]+bs[1]+bs[10]+bs[11]+bs[9]+bs[16]+bs[18]+bs[26]+bs[27]+bs[25]+bs[31]+bs[30]+bs[41]+bs[47]
bs_sum['financial_liability'] = bs[53]+bs[54]+bs[55]+bs[56]+bs[52]+bs[58]+bs[62]+bs[61]+bs[65]+bs[68]+bs[74]+bs[75]+bs[76]+bs[73]+bs[81]+bs[85]+bs[86]+bs[84]+bs[89]+bs[88]
bs_sum['equity'] = bs[95]+bs[94]+bs[98]+bs[97]+bs[99]+bs[101]+bs[103]+bs[105]
# 计算资本支出
bs_sum['operat_net_assets'] = bs_sum['operat_current'] + bs_sum['operat_non_curr'] - bs_sum['operat_liability']
bs_sum['capital_expense'] = bs_sum['operat_net_assets'] - bs_sum['operat_net_assets'].shift(-1)

bs_sum = bs_sum.head(5)   # 只取过去5年数据

total_shares = bs[94]    # 从资产负债表的‘实收资本’中获取流通股数量。疑问:数据中标明是金额万元，但核对后确认是流通股数量

In [9]:
bs_sum    # 输出整理完后的资产负债表

,operat_current,operat_non_curr,operat_liability,financial_assets,financial_liability,equity,operat_net_assets,capital_expense
2021-12-31,8709450.0,34280458.0,2252210.0,3233668.0,1286922.0,19190169.0,40737698.0,3379806.0
2020-12-31,7447513.0,31199939.0,1289560.0,3356807.0,1098629.0,16819758.0,37357892.0,3430156.0
2019-12-31,6801131.0,29149102.0,2022497.0,2582363.0,1224511.0,14217694.0,33927736.0,2125349.0
2018-12-31,5853693.0,27861927.0,1913233.0,1351835.0,1068405.0,11630762.0,31802387.0,3237326.0
2017-12-31,3466429.0,26672810.0,1574178.0,1178644.0,1443670.0,9167292.0,28565061.0,1106694.0


In [10]:
# 整理利润表
pl_sum = pd.DataFrame()

pl_sum['total_revenu'] = pl['营业总收入(万元)']
pl_sum['growth_rate'] = (pl['营业总收入(万元)'] - pl['营业总收入(万元)'].shift(-1)) / pl['营业总收入(万元)'].shift(-1)     # 计算历史每年销售增长率
pl_sum['intre_expen'] = pl['利息支出(万元)']
pl_sum['total_profit'] = pl['利润总额(万元)']
pl_sum['net_profit'] = pl['净利润(万元)']
pl_sum['income_tax'] = pl['所得税费用(万元)']
pl_sum['tax_rate'] = pl['所得税费用(万元)'] / pl['利润总额(万元)']    # 计算企业所得税率

pl_sum.drop('报告日期', inplace=True)    # 删除第一行
pl_sum = pl_sum.head(5)    # 提取历史5年数据

In [11]:
pl_sum     # 输出整理完后的损益表

,total_revenu,growth_rate,intre_expen,total_profit,net_profit,income_tax,tax_rate
2021-12-31,16795266.0,-0.047038,0.0,4411620.0,3416584.0,995036.0,0.225549
2020-12-31,17624268.0,0.122348,0.0,4710792.0,3637018.0,1073774.0,0.227939
2019-12-31,15703033.0,0.222953,0.0,4455685.0,3435201.0,1020484.0,0.229030
2018-12-31,12840263.0,0.704969,0.0,3962920.0,3063601.0,899318.0,0.226933
2017-12-31,7531082.0,0.346473,0.0,2122876.0,1642873.0,480002.0,0.226109


In [12]:
# 整理现金流量表
cf_sum = pd.DataFrame()
# 提取和资本支出相关项目数据
cf_sum['depr_fixed'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']    # 注意column的标题有一个前置空格
cf_sum['intang_amort'] = cf[' 无形资产摊销(万元)']
cf_sum['deferred'] = cf[' 长期待摊费用摊销(万元)']
# cf_sum['depreciation'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']+cf[' 无形资产摊销(万元)']+cf[' 长期待摊费用摊销(万元)']

cf_sum.drop('报告日期', inplace=True)    # 删除第一行
cf_sum = cf_sum.head(5)

In [13]:
cf_sum    # 输出整理完的现金流量表

,depr_fixed,intang_amort,deferred
2021-12-31,535191.0,60485.0,0.0
2020-12-31,498444.0,44512.0,0.0
2019-12-31,493163.0,33934.0,0.0
2018-12-31,464730.0,30611.0,0.0
2017-12-31,459028.0,24290.0,0.0


In [14]:
# 汇总历史估值数据
valuation = pd.DataFrame()

valuation['profit_after_tax'] = pl_sum['net_profit'] + pl_sum['intre_expen'] * (1 - pl_sum['tax_rate'] )
valuation['depreciation'] = cf_sum['depr_fixed'] + cf_sum['intang_amort'] + cf_sum['deferred']
valuation['capital_expense'] = bs_sum['capital_expense']
valuation['net_cash_flow'] = valuation['profit_after_tax'] + valuation['depreciation'] - valuation['capital_expense']

In [15]:
valuation

,profit_after_tax,depreciation,capital_expense,net_cash_flow
2021-12-31,3416584.0,595676.0,3379806.0,632454.0
2020-12-31,3637018.0,542956.0,3430156.0,749818.0
2019-12-31,3435201.0,527097.0,2125349.0,1836949.0
2018-12-31,3063601.0,495341.0,3237326.0,321616.0
2017-12-31,1642873.0,483318.0,1106694.0,1019497.0


In [16]:
discount_rate = 0.12    # 折现率，仅供参考
continue_rate = 0.05    # 永续经营增长率，通常固定为0.05

# 估算收入的增长率
growth_rate_max = 1+ pl_sum['growth_rate'].max()    # 最大
growth_rate_mean = 1 + pl_sum['growth_rate'].mean()    # 平均
growth_rate_min = 1 + pl_sum['growth_rate'].min()    # 最小
# growth_rate3 = np.percentile(pl_sum['growth_rate'], 75)    # 75分位

growth_rate = growth_rate_mean    # 增长率的选择可以调节
growth_rate_continue = 1.05


# 未来5年的增长比例，第6个是永续经营的比例
a = np.logspace(1,5,5, base=growth_rate)
growth_rates = np.append(a, a[4] * growth_rate_continue)
growth_rates


array([1.26994116, 1.61275054, 2.04809829, 2.60096432, 3.30307164,
       3.46822522])

In [17]:
# 预测未来收入，只和增长率相关
revenu_forecast = pl_sum.iloc[0, 0] * growth_rates
revenu_forecast

array([21328999.55153494, 27086574.38765053, 34398355.64181097,
       43683887.59414264, 55475966.79354594, 58249765.13322324])

In [18]:
# 预测未来净利润， 只和增长率相关
profit_forecast = valuation.iloc[0,0] * growth_rates
profit_forecast

array([ 4338860.64107478,  5510097.70655949,  6997499.86169443,
        8886413.07687215, 11285221.7125564 , 11849482.79818423])

In [19]:
# 预测未来5年资本支出
bs_rate = bs_sum.sum() / pl_sum['total_revenu'].sum()    # 各资本项占销售收入的历史比率
bs_rate

operat_current         0.457887
operat_non_curr        2.115987
operat_liability       0.128404
financial_assets       0.166019
financial_liability    0.086846
equity                 1.007543
operat_net_assets      2.445470
capital_expense        0.188376
dtype: float64

In [20]:
bs_forecast0 = bs_rate.head(5) * revenu_forecast[0]    # 基于未来的销售收入预测第一年bs项，只提取前5项

# b = np.logspace(0,4,5, base=growth_rate_max)
# np.append(b, b[4] * 1.05)

In [21]:
#np.expand_dims(b, 0).repeat(8, axis=0)
#np.expand_dims(bs_forecast.T, 0).repeat(5, axis=1)
bs_last = bs_sum.iloc[0].head(5)

In [22]:
# 分步计算第二至五年和第六个永续经营的资本支出项
bs_forecast1 = bs_forecast0 * growth_rate
bs_forecast2 = bs_forecast1 * growth_rate
bs_forecast3 = bs_forecast2 * growth_rate
bs_forecast4 = bs_forecast3 * growth_rate
bs_forecast5 = bs_forecast4 * growth_rate_continue

In [23]:
operate = pd.concat([bs_last, bs_forecast0, bs_forecast1, bs_forecast2, bs_forecast3, bs_forecast4, bs_forecast5], axis=1)
operate

,2021-12-31,0,1,2,3,4,5
operat_current,8709450.0,9.766263e+06,1.240258e+07,1.575055e+07,2.000227e+07,2.540170e+07,2.667179e+07
operat_non_curr,34280458.0,4.513190e+07,5.731485e+07,7.278649e+07,9.243456e+07,1.173865e+08,1.232558e+08
operat_liability,2252210.0,2.738722e+06,3.478016e+06,4.416876e+06,5.609172e+06,7.123319e+06,7.479484e+06
financial_assets,3233668.0,3.541016e+06,4.496881e+06,5.710775e+06,7.252348e+06,9.210055e+06,9.670558e+06
financial_liability,1286922.0,1.852345e+06,2.352369e+06,2.987371e+06,3.793785e+06,4.817884e+06,5.058778e+06


In [24]:
# 计算当年净负债，供NPV扣减净负债时用
net_liability = operate.iloc[4,0]-operate.iloc[3,0]    # Pandas的iloc函数只看行列的序号，不看行列的标号
net_liability

-1946746.0

In [25]:
# 计算经营性净资产
net_operate = operate.iloc[0]+operate.iloc[1]-operate.iloc[2]

In [28]:
# 计算未来5年和永续经营的资本支出
capex_forecast = net_operate - net_operate.shift(1) 
# 提取前6项
capex_forecast = np.array(capex_forecast)[1:7]

In [27]:
capex_forecast

array([11421738.70901342, 14079978.74143925, 17880744.50632572,
       22707493.38275038, 28837180.43984333,  6783241.6889686 ])

In [29]:
# 预测5年的折旧
c = np.array(valuation['depreciation'])
c = c[0:3]
depre = c[::-1]

In [30]:
# g = np.full(5, growth_rate_max)    # 用指定增长率构造一个5元数组
# growth_rates = np.append(growth_rates, growth_rate_continue)

for i in np.arange(3,9):    # 构造整数序列
    d = (depre[(i-3)] + depre[(i-2)] + depre[(i-1)]) / 3     # 前三年的平均折旧金额
    d = d * growth_rate
    depre = np.append(depre, d)

depre_forecast = depre[3:9]    # 截取最后6项, 未来5年加永续经营

In [31]:
depre_forecast

array([ 705125.93834435,  780488.03037883,  881038.59761258,
       1001836.50015853, 1127438.15163719, 1274306.89788041])

In [32]:
# 计算现金流量
forecast_net_cash = profit_forecast + depre_forecast - capex_forecast
ca1 = forecast_net_cash[0:5]    # 只计算今后5年
ca2 = np.concatenate(([0], ca1))    # 当年的现金流要设为0，适应numpy的npv函数设定
contin = forecast_net_cash[5] / (discount_rate - continue_rate)
ca = np.concatenate((ca2, [contin]))    # 添加永续经营现金流

In [35]:
# 计算净现值NPV
net_present_value = np.npv(discount_rate, ca)    # 利用numpy的npv函数计算净现值
estimated_stock_value = (net_present_value - net_liability) / total_shares    # 净现值扣除当前净负债

In [36]:
# 输出估值
estimated_stock_value[0]    # 最终估值

21.41224452281828

#### 2.财务指标

In [43]:
# 资产总计：资产增长率不低于10%
cap_growth_rate = (bs[51] - bs[51].shift(-1)) / bs[51].shift(-1)    # bs[51]资产合计
cap_growth_rate

2021-12-31    0.146995
2020-12-31    0.124152
2019-12-31    0.195455
2018-12-31    0.224367
2017-12-31    0.115314
2016-12-31    0.035287
2015-12-31    0.034505
2014-12-31    0.098380
2013-12-31    0.063651
2012-12-31    0.041904
2011-12-31    0.390512
2010-12-31    0.281310
2009-12-31    0.108539
2008-12-31    0.370209
2007-12-31    0.353365
2006-12-31         NaN
Name: 51, dtype: float64

In [44]:
# 负债合计： 负债占比不大于60%
liab_rate = bs[93] / bs[51]    # bs[93]负债合计
liab_rate

2021-12-31    0.167836
2020-12-31    0.162987
2019-12-31    0.203923
2018-12-31    0.221532
2017-12-31    0.247076
2016-12-31    0.266772
2015-12-31    0.301280
2014-12-31    0.319226
2013-12-31    0.368758
2012-12-31    0.415288
2011-12-31    0.442383
2010-12-31    0.410185
2009-12-31    0.378498
2008-12-31    0.405903
2007-12-31    0.627979
2006-12-31    0.613276
dtype: float64

In [45]:
# 应付-应受 ： 应付票据+应付账款+预收账款 - 应收票据+应收账款+预付账款
(bs[58] + bs[59] + bs[60]) - (bs[5] + bs[6] + bs[7])


2021-12-31    -472420.0
2020-12-31    -648438.0
2019-12-31    -503779.0
2018-12-31   -1008008.0
2017-12-31    -585884.0
2016-12-31    -181740.0
2015-12-31     -10036.0
2014-12-31      34203.0
2013-12-31    -315785.0
2012-12-31    -293701.0
2011-12-31    -556672.0
2010-12-31    -215792.0
2009-12-31     139407.0
2008-12-31     -49647.0
2007-12-31    -147475.0
2006-12-31       8812.0
dtype: float64

In [46]:
# 应受占比: 小于3%
bs[6] / bs[51]

2021-12-31    0.010312
2020-12-31    0.005993
2019-12-31    0.007124
2018-12-31    0.008242
2017-12-31    0.008676
2016-12-31    0.005978
2015-12-31    0.003990
2014-12-31    0.003259
2013-12-31    0.003855
2012-12-31    0.002632
2011-12-31    0.004089
2010-12-31    0.008268
2009-12-31    0.009385
2008-12-31    0.010499
2007-12-31    0.007394
2006-12-31    0.017119
dtype: float64

In [47]:
# 固定资产占比： 不大于40%，固定资产+在建工程+工程物资
(bs[36] + bs[37] + bs[38]) / bs[51]

2021-12-31    0.320126
2020-12-31    0.335345
2019-12-31    0.364120
2018-12-31    0.426479
2017-12-31    0.518749
2016-12-31    0.592161
2015-12-31    0.608116
2014-12-31    0.612623
2013-12-31    0.608296
2012-12-31    0.606692
2011-12-31    0.585380
2010-12-31    0.652615
2009-12-31    0.677723
2008-12-31    0.599693
2007-12-31    0.705068
2006-12-31    0.746423
dtype: float64

In [48]:
# 存货占比
bs[19] / bs[51]

2021-12-31    0.042928
2020-12-31    0.034839
2019-12-31    0.031165
2018-12-31    0.040273
2017-12-31    0.038522
2016-12-31    0.041534
2015-12-31    0.040064
2014-12-31    0.042796
2013-12-31    0.039666
2012-12-31    0.046142
2011-12-31    0.052122
2010-12-31    0.041269
2009-12-31    0.046071
2008-12-31    0.043975
2007-12-31    0.050339
2006-12-31    0.055497
dtype: float64

In [49]:
# 商誉占比
bs[45] / bs[51]

2021-12-31    0.003800
2020-12-31    0.002866
2019-12-31    0.002877
2018-12-31    0.003440
2017-12-31    0.004042
2016-12-31    0.004508
2015-12-31    0.004384
2014-12-31    0.003663
2013-12-31    0.003220
2012-12-31    0.002427
2011-12-31    0.000949
2010-12-31    0.000267
2009-12-31    0.000342
2008-12-31    0.000379
2007-12-31    0.000519
2006-12-31    0.000703
dtype: float64

In [62]:
pl.drop('报告日期', inplace=True)    # 删除第一行
pl.drop(pl.tail(1).index, inplace=True)    # 删除最后一行
pl

报告日期,营业总收入(万元),营业收入(万元),利息收入(万元),已赚保费(万元),手续费及佣金收入(万元),房地产销售收入(万元),其他业务收入(万元),营业总成本(万元),营业成本(万元),利息支出(万元),...,非流动资产处置损失(万元),利润总额(万元),所得税费用(万元),未确认投资损失(万元),净利润(万元),归属于母公司所有者的净利润(万元),被合并方在合并前实现净利润(万元),少数股东损益(万元),基本每股收益,稀释每股收益
2021-12-31,16795266.0,16795266.0,0.0,0.0,0.0,0.0,0.0,12791903.0,11818122.0,0.0,...,0.0,4411620.0,995036.0,0.0,3416584.0,3326707.0,0.0,89877.0,6.28,6.28
2020-12-31,17624268.0,17624268.0,0.0,0.0,0.0,0.0,0.0,13359860.0,12484763.0,0.0,...,0.0,4710792.0,1073774.0,0.0,3637018.0,3512969.0,0.0,124049.0,6.63,6.63
2019-12-31,15703033.0,15703033.0,0.0,0.0,0.0,0.0,0.0,11416990.0,10476009.0,0.0,...,0.0,4455685.0,1020484.0,0.0,3435201.0,3359276.0,0.0,75925.0,6.34,6.34
2018-12-31,12840263.0,12840263.0,0.0,0.0,0.0,0.0,0.0,8997946.0,8123003.0,0.0,...,0.0,3962920.0,899318.0,0.0,3063601.0,2981428.0,0.0,82173.0,5.63,5.63
2017-12-31,7531082.0,7531082.0,0.0,0.0,0.0,0.0,0.0,5708195.0,4888773.0,0.0,...,0.0,2122876.0,480002.0,0.0,1642873.0,1585467.0,0.0,57406.0,2.99,2.99
2016-12-31,5593190.0,5593190.0,0.0,0.0,0.0,0.0,0.0,4557713.0,3776996.0,0.0,...,693.0,1165321.0,270256.0,0.0,895064.0,852992.0,0.0,42073.0,1.61,1.61
2015-12-31,5097604.0,5097604.0,0.0,0.0,0.0,0.0,0.0,4416517.0,3688786.0,0.0,...,1041.0,1003940.0,241145.0,0.0,762795.0,751639.0,0.0,11157.0,1.42,1.42
2014-12-31,6075850.0,6075850.0,0.0,0.0,0.0,0.0,0.0,4693493.0,4026433.0,0.0,...,938.0,1488281.0,329521.0,0.0,1158760.0,1099302.0,0.0,59457.0,2.07,2.07
2013-12-31,5526168.0,5526168.0,0.0,0.0,0.0,0.0,0.0,4352434.0,3701808.0,0.0,...,856.0,1263127.0,281981.0,0.0,981145.0,938016.0,0.0,43129.0,1.77,1.77
2012-12-31,4576620.0,4576620.0,0.0,0.0,0.0,0.0,0.0,3875722.0,3306192.0,0.0,...,561.0,808782.0,162559.0,0.0,646223.0,630759.0,0.0,15464.0,1.19,1.19


In [54]:
# 毛利率
(pl['营业收入(万元)'] - pl['营业成本(万元)']) / pl['营业收入(万元)']

2021-12-31     0.296342
2020-12-31     0.291615
2019-12-31     0.332867
2018-12-31     0.367380
2017-12-31     0.350854
2016-12-31     0.324715
2015-12-31     0.276369
2014-12-31     0.337305
2013-12-31     0.330131
2012-12-31     0.277591
2011-12-31     0.398899
2010-12-31     0.320839
2009-12-31     0.285327
2008-12-31     0.249156
2007-12-31     0.311587
2006-12-31     0.284954
Unnamed: 17         NaN
dtype: float64

In [56]:
# 期间费率
(pl['销售费用(万元)'] + pl['管理费用(万元)'] + pl['研发费用(万元)'] + pl['财务费用(万元)']) / pl['营业收入(万元)']    

2021-12-31     0.050576
2020-12-31     0.042341
2019-12-31     0.050989
2018-12-31     0.055157
2017-12-31     0.096231
2016-12-31     0.120805
2015-12-31     0.134420
2014-12-31     0.104202
2013-12-31     0.109457
2012-12-31     0.119190
2011-12-31     0.087921
2010-12-31     0.095096
2009-12-31     0.110097
2008-12-31     0.122126
2007-12-31     0.135658
2006-12-31     0.132482
Unnamed: 17         NaN
dtype: float64

In [63]:
# ROE: 归属于母公司所有者的净利润 / 归属于母公司股东权益合计
pl['归属于母公司所有者的净利润(万元)'] / bs[104]

2021-12-31    0.181109
2020-12-31    0.217088
2019-12-31    0.244557
2018-12-31    0.264572
2017-12-31    0.177333
2016-12-31    0.111344
2015-12-31    0.106628
2014-12-31    0.166016
2013-12-31    0.167151
2012-12-31    0.128986
2011-12-31    0.258474
2010-12-31    0.176309
2009-12-31    0.123241
2008-12-31    0.105136
2007-12-31    0.225118
2006-12-31    0.206066
dtype: float64

In [67]:
cf.drop('报告日期', inplace=True)    # 删除第一行
cf.drop(cf.tail(1).index, inplace=True)    # 删除最后一行


报告日期,销售商品、提供劳务收到的现金(万元),客户存款和同业存放款项净增加额(万元),向中央银行借款净增加额(万元,向其他金融机构拆入资金净增加额(万元),收到原保险合同保费取得的现金(万元),收到再保险业务现金净额(万元),保户储金及投资款净增加额(万元),处置交易性金融资产净增加额(万元),收取利息、手续费及佣金的现金(万元),拆入资金净增加额(万元),...,其他(万元),经营活动产生现金流量净额(万元),债务转为资本(万元),一年内到期的可转换公司债券(万元),融资租入固定资产(万元),现金的期末余额(万元),现金的期初余额(万元),现金等价物的期末余额(万元),现金等价物的期初余额(万元),现金及现金等价物的净增加额(万元)
2021-12-31,20064530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3390053.0,0.0,0.0,0.0,1737751.0,1679931.0,0.0,0.0,57821.0
2020-12-31,21298942.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3479721.0,0.0,0.0,0.0,1667638.0,2201414.0,0.0,0.0,-533776.0
2019-12-31,19605125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4073820.0,0.0,0.0,0.0,2201414.0,985767.0,0.0,0.0,1215647.0
2018-12-31,16378360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3605897.0,0.0,0.0,0.0,985767.0,1042893.0,0.0,0.0,-57126.0
2017-12-31,9265857.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1736303.0,0.0,0.0,0.0,1042893.0,579957.0,0.0,0.0,462937.0
2016-12-31,6938337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1319675.0,0.0,0.0,0.0,579957.0,428503.0,0.0,0.0,151453.0
2015-12-31,6466593.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,990817.0,0.0,0.0,0.0,428503.0,1251212.0,0.0,0.0,-822709.0
2014-12-31,8133890.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1765449.0,0.0,0.0,0.0,1251212.0,651893.0,0.0,0.0,599319.0
2013-12-31,7083703.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1519855.0,0.0,0.0,0.0,651893.0,811097.0,0.0,0.0,-159204.0
2012-12-31,6036199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1150864.0,0.0,0.0,0.0,811097.0,774719.0,0.0,0.0,36379.0


In [72]:
# 营收占比
cf[' 销售商品、提供劳务收到的现金(万元)'] / pl['营业总收入(万元)']

2021-12-31    1.194654
2020-12-31    1.208501
2019-12-31    1.248493
2018-12-31    1.275547
2017-12-31    1.230349
2016-12-31    1.240497
2015-12-31    1.268555
2014-12-31    1.338725
2013-12-31    1.281847
2012-12-31    1.318921
2011-12-31    1.176253
2010-12-31    1.172361
2009-12-31    1.321778
2008-12-31    1.253054
2007-12-31    1.143958
2006-12-31    1.092276
dtype: float64

In [75]:
(cf[' 经营活动产生的现金流量净额(万元)'] - cf[' 经营活动产生的现金流量净额(万元)'].shift(-1)) / cf[' 经营活动产生的现金流量净额(万元)'].shift(-1)

2021-12-31   -0.025769
2020-12-31   -0.145833
2019-12-31    0.129766
2018-12-31    1.076767
2017-12-31    0.315705
2016-12-31    0.331906
2015-12-31   -0.438773
2014-12-31    0.161590
2013-12-31    0.320621
2012-12-31    0.096917
2011-12-31    0.745635
2010-12-31   -0.144969
2009-12-31    0.334762
2008-12-31    0.973303
2007-12-31   -0.125763
2006-12-31         NaN
Name:  经营活动产生的现金流量净额(万元), dtype: float64

In [74]:
# 固定资产支付现金占比
cf[' 购建固定资产、无形资产和其他长期资产所支付的现金(万元)'] / cf[' 经营活动产生的现金流量净额(万元)']

2021-12-31    0.448439
2020-12-31    0.286523
2019-12-31    0.217833
2018-12-31    0.131688
2017-12-31    0.210570
2016-12-31    0.377479
2015-12-31    0.521527
2014-12-31    0.395001
2013-12-31    0.497519
2012-12-31    0.592170
2011-12-31    0.786337
2010-12-31    1.541829
2009-12-31    1.138404
2008-12-31    0.978378
2007-12-31    2.092401
2006-12-31    1.144369
dtype: float64

In [ ]:
pl['基本每股收益']

In [77]:
cf[[' 投资活动产生的现金流量净额(万元)',' 筹资活动产生的现金流量净额(万元)']]

报告日期,投资活动产生的现金流量净额(万元),筹资活动产生的现金流量净额(万元)
2021-12-31,-2166696.0,-1160447.0
2020-12-31,-2677288.0,-1326202.0
2019-12-31,-2068885.0,-791190.0
2018-12-31,-2566970.0,-1098000.0
2017-12-31,-520265.0,-749961.0
2016-12-31,-455225.0,-715095.0
2015-12-31,-1271927.0,-539532.0
2014-12-31,-485111.0,-678568.0
2013-12-31,-1247633.0,-428937.0
2012-12-31,-846966.0,-267092.0
